## Set Up

In [12]:
import pandas as pd
import calendar
import datetime

In [13]:
mta190504_raw = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_190504.txt")
mta190504_raw.describe()

,ENTRIES,EXITS
count,2.068570e+05,2.068570e+05
mean,4.057530e+07,3.319799e+07
std,2.082752e+08,1.926699e+08
min,0.000000e+00,0.000000e+00
25%,3.495460e+05,1.413340e+05
50%,2.176408e+06,1.241604e+06
75%,6.775342e+06,4.590174e+06
max,2.129343e+09,2.124127e+09


## Data Cleaning and Variable Creation

In [14]:
#Clean column names
mta190504_raw.columns = mta190504_raw.columns.str.strip()
mta190504_raw.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [15]:
# Go from raw DF to new DF that will have rows dropped
mta190504 = mta190504_raw.copy()

# Create variables:
# "time_hour" that simplifies the hour for later grouping purposes
mta190504["time_hour"] = pd.to_numeric(mta190504["TIME"].str[0:2])
# formatted date variable "DDATE"
mta190504['DDATE']=[datetime.datetime.strptime(x, '%m/%d/%Y') for x in mta190504['DATE']]
# formatted time variable "DTIME"
mta190504['DTIME']=[format(datetime.datetime.strptime(x, '%H:%M:%S'),"%H:%M:%S") for x in mta190504['TIME']]
# formatted day of the week variable "DDAY"
mta190504['DDAY']=[calendar.day_name[datetime.datetime.weekday(x)] for x in mta190504['DDATE']]

In [16]:
# Sort before grouping
mta190504.sort_values(["C/A","UNIT","SCP","STATION","LINENAME","DIVISION","DATE","TIME","DESC"])
# Create difference columns to calculate difference in entries and exits between the row and the row before (aka the time before)
mta190504["entries_diff"] = mta190504.groupby(["C/A","UNIT","SCP","STATION","LINENAME","DIVISION"]).ENTRIES.diff()
mta190504["exits_diff"] = mta190504.groupby(["C/A","UNIT","SCP","STATION","LINENAME","DIVISION"]).EXITS.diff()
mta190504["entries-exits"] = mta190504["entries_diff"] - mta190504["exits_diff"]
mta190504["entries+exits"] = mta190504["entries_diff"] + mta190504["exits_diff"]
mta190504.describe()

,ENTRIES,EXITS,time_hour,entries_diff,exits_diff,entries-exits,entries+exits
count,2.068570e+05,2.068570e+05,206857.000000,2.020060e+05,2.020060e+05,2.020060e+05,2.020060e+05
mean,4.057530e+07,3.319799e+07,10.517256,2.075907e+03,7.433828e+03,-5.357922e+03,9.509735e+03
std,2.082752e+08,1.926699e+08,6.806081,3.508686e+06,3.415174e+06,1.754742e+06,6.698467e+06
min,0.000000e+00,0.000000e+00,0.000000,-8.392517e+08,-3.843496e+08,-6.714099e+08,-1.007094e+09
25%,3.495460e+05,1.413340e+05,4.000000,9.000000e+00,8.000000e+00,-2.500000e+01,2.700000e+01
50%,2.176408e+06,1.241604e+06,10.000000,7.200000e+01,5.200000e+01,1.000000e+00,1.680000e+02
75%,6.775342e+06,4.590174e+06,16.000000,2.390000e+02,1.660000e+02,9.700000e+01,4.550000e+02
max,2.129343e+09,2.124127e+09,23.000000,1.126553e+09,1.078346e+09,4.820644e+07,2.204899e+09


In [17]:
#keep only rows with positive entries_diff, exits_diff, and ENTRIES
mta190504= mta190504[mta190504.entries_diff > 0]
mta190504 = mta190504[mta190504.exits_diff > 0]
mta190504 = mta190504[mta190504.ENTRIES > 0]
mta190504 = mta190504[mta190504.entries_diff < 10**7]
mta190504 = mta190504[mta190504.exits_diff < 10**7]
mta190504.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165744 entries, 1 to 206772
Data columns (total 19 columns):
C/A              165744 non-null object
UNIT             165744 non-null object
SCP              165744 non-null object
STATION          165744 non-null object
LINENAME         165744 non-null object
DIVISION         165744 non-null object
DATE             165744 non-null object
TIME             165744 non-null object
DESC             165744 non-null object
ENTRIES          165744 non-null int64
EXITS            165744 non-null int64
time_hour        165744 non-null int64
DDATE            165744 non-null datetime64[ns]
DTIME            165744 non-null object
DDAY             165744 non-null object
entries_diff     165744 non-null float64
exits_diff       165744 non-null float64
entries-exits    165744 non-null float64
entries+exits    165744 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(3), object(11)
memory usage: 25.3+ MB


In [18]:
# Checks
# No negative values in entries_diff or exits_diff
mta190504.describe()

,ENTRIES,EXITS,time_hour,entries_diff,exits_diff,entries-exits,entries+exits
count,1.657440e+05,1.657440e+05,165744.000000,165744.000000,1.657440e+05,1.657440e+05,1.657440e+05
mean,2.717613e+07,2.016616e+07,10.914603,222.023162,3.212516e+02,-9.922846e+01,5.432748e+02
std,1.592185e+08,1.345442e+08,6.713721,1045.876071,1.470697e+04,1.369693e+04,1.572171e+04
min,2.000000e+00,9.000000e+00,0.000000,1.000000,1.000000e+00,-1.281387e+06,2.000000e+00
25%,6.766368e+05,3.798185e+05,5.000000,31.000000,2.400000e+01,-4.000000e+01,8.200000e+01
50%,2.814699e+06,1.657634e+06,12.000000,116.000000,7.800000e+01,1.200000e+01,2.460000e+02
75%,7.036347e+06,5.019779e+06,17.000000,290.000000,2.020000e+02,1.340000e+02,5.330000e+02
max,2.115816e+09,2.037805e+09,23.000000,94944.000000,1.376261e+06,6.345000e+03,1.471135e+06


In [19]:
#create intervals: 01-04;05-08;09-12;13-16;17-20;21-00
#start with 01 instead of 00, assuming that most audits are on the hour so first element is inclusive
#and last element is exclusive
def time_interval(x):
    if x in [1,2,3,4]:
        return "01:00-04:59"
    elif x in [5,6,7,8]:
        return "05:00-08:59"
    elif x in [9,10,11,12]:
        return "09:00-12:59"
    elif x in [13,14,15,16]:
        return "13:00-16:59"
    elif x in [17,18,19,20]:
        return "17:00-20:59"
    elif x in [21,22,23,0]:
        return "21:00-00:59"

In [20]:
# create time period category "time_cat"
mta190504["time_cat"] = mta190504["time_hour"].apply(time_interval)
mta190504.head(50)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,time_hour,DDATE,DTIME,DDAY,entries_diff,exits_diff,entries-exits,entries+exits,time_cat
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,04:00:00,REGULAR,7035269,2384840,4,2019-04-27,04:00:00,Saturday,20.0,7.0,13.0,27.0,01:00-04:59
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,08:00:00,REGULAR,7035292,2384875,8,2019-04-27,08:00:00,Saturday,23.0,35.0,-12.0,58.0,05:00-08:59
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,12:00:00,REGULAR,7035392,2384951,12,2019-04-27,12:00:00,Saturday,100.0,76.0,24.0,176.0,09:00-12:59
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,16:00:00,REGULAR,7035651,2385020,16,2019-04-27,16:00:00,Saturday,259.0,69.0,190.0,328.0,13:00-16:59
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,20:00:00,REGULAR,7035930,2385070,20,2019-04-27,20:00:00,Saturday,279.0,50.0,229.0,329.0,17:00-20:59
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2019,00:00:00,REGULAR,7036100,2385087,0,2019-04-28,00:00:00,Sunday,170.0,17.0,153.0,187.0,21:00-00:59
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2019,04:00:00,REGULAR,7036119,2385088,4,2019-04-28,04:00:00,Sunday,19.0,1.0,18.0,20.0,01:00-04:59
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2019,08:00:00,REGULAR,7036125,2385103,8,2019-04-28,08:00:00,Sunday,6.0,15.0,-9.0,21.0,05:00-08:59
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2019,12:00:00,REGULAR,7036197,2385155,12,2019-04-28,12:00:00,Sunday,72.0,52.0,20.0,124.0,09:00-12:59
10,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2019,16:00:00,REGULAR,7036372,2385198,16,2019-04-28,16:00:00,Sunday,175.0,43.0,132.0,218.0,13:00-16:59


In [21]:
# For outlier entries and exit differences, replace with imputed mean for the remaining valid entries
# INSERT SHREYAK'S CODE

## Output grouped data

In [22]:
# Group by station
mta_station = mta190504.groupby(["STATION"])[["entries_diff","exits_diff","entries+exits"]].sum().sort_values(["entries+exits","entries_diff","exits_diff"], ascending=False)
mta_station

,entries_diff,exits_diff,entries+exits
STATION,,,
TWENTY THIRD ST,1854775.0,26142573.0,27997348.0
34 ST-PENN STA,1023199.0,922432.0,1945631.0
GRD CNTRL-42 ST,881117.0,769767.0,1650884.0
34 ST-HERALD SQ,703365.0,661376.0,1364741.0
14 ST-UNION SQ,650999.0,613366.0,1264365.0
TIMES SQ-42 ST,614463.0,579138.0,1193601.0
23 ST,666245.0,465315.0,1131560.0
FULTON ST,586908.0,488472.0,1075380.0
42 ST-PORT AUTH,565110.0,482997.0,1048107.0


In [23]:
# Group by station and date
mta_station_date = mta190504.groupby(["STATION","DDATE"])[["entries_diff","exits_diff","entries+exits"]].sum().sort_values(["entries+exits","entries_diff","exits_diff"], ascending=False)
mta_station_date

entries_diff  exits_diff  entries+exits
STATION         DDATE                                              
TWENTY THIRD ST 2019-04-28      570806.0   8238772.0      8809578.0
                2019-04-30      484237.0   6882493.0      7366730.0
                2019-04-29      388084.0   5502804.0      5890888.0
                2019-04-27      381941.0   5491754.0      5873695.0
34 ST-PENN STA  2019-05-02      175657.0    157005.0       332662.0
                2019-05-01      175541.0    156632.0       332173.0
                2019-04-30      172612.0    150738.0       323350.0
                2019-05-03      167620.0    155012.0       322632.0
                2019-04-29      166379.0    144031.0       310410.0
GRD CNTRL-42 ST 2019-05-01      161753.0    137785.0       299538.0
                2019-05-02      160529.0    138224.0       298753.0
                2019-04-30      159507.0    136490.0       295997.0
                2019-05-03      154908.0    131888.0       286796.0
                2019-04-29      147482.0    129125.0       276607.0
34 ST-HERALD SQ 2019-05-03      121247.0    113413.0       234660.0
                2019-05-02      119941.0    113678.0       233619.0
                2019-05-01      119835.0    111959.0       231794.0
                2019-04-30      117319.0    108088.0       225407.0
                2019-04-29      108935.0    102544.0       211479.0
14 ST-UNION SQ  2019-05-01      109834.0    101138.0       210972.0
                2019-05-03      109130.0    101029.0       210159.0
                2019-05-02      109420.0    100219.0       209639.0
                2019-04-30      105308.0     99039.0       204347.0
TIMES SQ-42 ST  2019-05-01      106673.0     96022.0       202695.0
                2019-05-02      104607.0     96313.0       200920.0
                2019-05-03      104220.0     96103.0       200323.0
FULTON ST       2019-05-02      107574.0     89670.0       197244.0
                2019-05-01      106958.0     89317.0       196275.0
23 ST           2019-05-01      117632.0     75842.0       193474.0
14 ST-UNION SQ  2019-04-29       98358.0     93872.0       192230.0
...                                  ...         ...            ...
BEACH 105 ST    2019-05-01         236.0       259.0          495.0
                2019-04-30         232.0       252.0          484.0
BROAD CHANNEL   2019-05-02         378.0        72.0          450.0
GRANT AV        2019-04-27          93.0       355.0          448.0
BROAD CHANNEL   2019-05-03         340.0        85.0          425.0
                2019-04-30         335.0        69.0          404.0
                2019-05-01         309.0        82.0          391.0
NEWARK C        2019-04-28         341.0        47.0          388.0
BROAD CHANNEL   2019-04-29         289.0        79.0          368.0
OZONE PK LEFFRT 2019-04-27         106.0       260.0          366.0
BEACH 98 ST     2019-04-27         157.0       133.0          290.0
PATH WTC 2      2019-04-27         202.0        38.0          240.0
80 ST           2019-04-27          24.0       192.0          216.0
PATH WTC 2      2019-04-28         182.0        28.0          210.0
BEACH 98 ST     2019-04-28          98.0        91.0          189.0
BROAD CHANNEL   2019-04-27          70.0        74.0          144.0
AQUEDUCT RACETR 2019-04-27         133.0        10.0          143.0
BROAD CHANNEL   2019-04-28          65.0        63.0          128.0
BEACH 105 ST    2019-04-27          53.0        74.0          127.0
ROCKAWAY BLVD   2019-04-27          42.0        78.0          120.0
157 ST          2019-04-27          52.0        64.0          116.0
AQUEDUCT RACETR 2019-04-28          52.0        35.0           87.0
88 ST           2019-04-27          25.0        59.0           84.0
BEACH 105 ST    2019-04-28          36.0        46.0           82.0
AQUEDUCT N.COND 2019-04-27          16.0        57.0           73.0
ST. GEORGE      2019-04-28          43.0         2.0           45.0
207 ST          

In [24]:
# Group by station and day of week
mta_station_day = mta190504.groupby(["STATION","DDAY"])[["entries_diff","exits_diff","entries+exits"]].sum().sort_values(["entries+exits","entries_diff","exits_diff"], ascending=False)
mta_station_day

entries_diff  exits_diff  entries+exits
STATION         DDAY                                              
TWENTY THIRD ST Sunday         570806.0   8238772.0      8809578.0
                Tuesday        484237.0   6882493.0      7366730.0
                Monday         388084.0   5502804.0      5890888.0
                Saturday       381941.0   5491754.0      5873695.0
34 ST-PENN STA  Thursday       175657.0    157005.0       332662.0
                Wednesday      175541.0    156632.0       332173.0
                Tuesday        172612.0    150738.0       323350.0
                Friday         167620.0    155012.0       322632.0
                Monday         166379.0    144031.0       310410.0
GRD CNTRL-42 ST Wednesday      161753.0    137785.0       299538.0
                Thursday       160529.0    138224.0       298753.0
                Tuesday        159507.0    136490.0       295997.0
                Friday         154908.0    131888.0       286796.0
                Monday         147482.0    129125.0       276607.0
34 ST-HERALD SQ Friday         121247.0    113413.0       234660.0
                Thursday       119941.0    113678.0       233619.0
                Wednesday      119835.0    111959.0       231794.0
                Tuesday        117319.0    108088.0       225407.0
                Monday         108935.0    102544.0       211479.0
14 ST-UNION SQ  Wednesday      109834.0    101138.0       210972.0
                Friday         109130.0    101029.0       210159.0
                Thursday       109420.0    100219.0       209639.0
                Tuesday        105308.0     99039.0       204347.0
TIMES SQ-42 ST  Wednesday      106673.0     96022.0       202695.0
                Thursday       104607.0     96313.0       200920.0
                Friday         104220.0     96103.0       200323.0
FULTON ST       Thursday       107574.0     89670.0       197244.0
                Wednesday      106958.0     89317.0       196275.0
23 ST           Wednesday      117632.0     75842.0       193474.0
14 ST-UNION SQ  Monday          98358.0     93872.0       192230.0
...                                 ...         ...            ...
BEACH 105 ST    Wednesday         236.0       259.0          495.0
                Tuesday           232.0       252.0          484.0
BROAD CHANNEL   Thursday          378.0        72.0          450.0
GRANT AV        Saturday           93.0       355.0          448.0
BROAD CHANNEL   Friday            340.0        85.0          425.0
                Tuesday           335.0        69.0          404.0
                Wednesday         309.0        82.0          391.0
NEWARK C        Sunday            341.0        47.0          388.0
BROAD CHANNEL   Monday            289.0        79.0          368.0
OZONE PK LEFFRT Saturday          106.0       260.0          366.0
BEACH 98 ST     Saturday          157.0       133.0          290.0
PATH WTC 2      Saturday          202.0        38.0          240.0
80 ST           Saturday           24.0       192.0          216.0
PATH WTC 2      Sunday            182.0        28.0          210.0
BEACH 98 ST     Sunday             98.0        91.0          189.0
BROAD CHANNEL   Saturday           70.0        74.0          144.0
AQUEDUCT RACETR Saturday          133.0        10.0          143.0
BROAD CHANNEL   Sunday             65.0        63.0          128.0
BEACH 105 ST    Saturday           53.0        74.0          127.0
ROCKAWAY BLVD   Saturday           42.0        78.0          120.0
157 ST          Saturday           52.0        64.0          116.0
AQUEDUCT RACETR Sunday             52.0        35.0           87.0
88 ST           Saturday           25.0        59.0           84.0
BEACH 105 ST    Sunday             36.0        46.0           82.0
AQUEDUCT N.COND Saturday           16.0        57.0           73.0
ST. GEORGE      Sunday             43.0         2.0           45.0
207 ST          Saturday            3.0        38.0           41.0
ST. GEO

In [25]:
# Group by station, day of week, and time of day
mta_station_day_time = mta190504.groupby(["STATION","DDAY","time_cat"])[["entries_diff","exits_diff","entries+exits"]].sum().sort_values(["entries+exits","entries_diff","exits_diff"], ascending=False)
mta_station_day_time

entries_diff  exits_diff  entries+exits
STATION         DDAY      time_cat                                            
TWENTY THIRD ST Tuesday   17:00-20:59       98621.0   1377162.0      1475783.0
                Monday    17:00-20:59       98952.0   1375547.0      1474499.0
                Tuesday   09:00-12:59       95714.0   1377954.0      1473668.0
                Monday    09:00-12:59       95462.0   1377769.0      1473231.0
                Tuesday   13:00-16:59       95494.0   1376928.0      1472422.0
                Monday    13:00-16:59       95768.0   1374993.0      1470761.0
                Tuesday   05:00-08:59       94997.0   1375028.0      1470025.0
                          01:00-04:59       95186.0   1374559.0      1469745.0
                Saturday  17:00-20:59       96052.0   1373243.0      1469295.0
                Monday    05:00-08:59       94887.0   1374314.0      1469201.0
                Sunday    17:00-20:59       95623.0   1373512.0      1469135.0
                          21:00-00:59       95584.0   1373260.0      1468844.0
                Saturday  21:00-00:59       95645.0   1373180.0      1468825.0
                Sunday    13:00-16:59       95082.0   1373609.0      1468691.0
                Saturday  13:00-16:59       95309.0   1372756.0      1468065.0
                Sunday    01:00-04:59       95139.0   1372792.0      1467931.0
                          09:00-12:59       94716.0   1372987.0      1467703.0
                Saturday  09:00-12:59       94809.0   1372506.0      1467315.0
                Sunday    05:00-08:59       94662.0   1372612.0      1467274.0
34 ST-PENN STA  Thursday  17:00-20:59       45736.0     61334.0       107070.0
                Wednesday 17:00-20:59       44860.0     62149.0       107009.0
                Friday    17:00-20:59       45080.0     59305.0       104385.0
                Tuesday   17:00-20:59       44018.0     59357.0       103375.0
                Monday    17:00-20:59       43261.0     59254.0       102515.0
                Tuesday   09:00-12:59       60710.0     33223.0        93933.0
                Wednesday 09:00-12:59       59399.0     33629.0        93028.0
                Thursday  09:00-12:59       58811.0     33829.0        92640.0
34 ST-HERALD SQ Thursday  17:00-20:59       50887.0     33167.0        84054.0
34 ST-PENN STA  Friday    09:00-12:59       51239.0     32590.0        83829.0
GRD CNTRL-42 ST Friday    17:00-20:59       50749.0     32729.0        83478.0
...                                             ...         ...            ...
BEACH 105 ST    Saturday  05:00-08:59           4.0         6.0           10.0
NEPTUNE AV      Saturday  05:00-08:59           7.0         2.0            9.0
BROAD CHANNEL   Monday    05:00-08:59           6.0         3.0            9.0
ST. GEORGE      Monday    21:00-00:59           6.0         3.0            9.0
BROAD CHANNEL   Monday    01:00-04:59           5.0         4.0            9.0
NEWARK HW BMEBE Sunday    01:00-04:59           5.0         4.0            9.0
LACKAWANNA      Sunday    05:00-08:59           2.0         7.0            9.0
PATH WTC 2      Saturday  05:00-08:59           1.0         8.0            9.0
RIT-MANHATTAN   Friday    05:00-08:59           7.0         1.0            8.0
RIT-ROOSEVELT   Saturday  05:00-08:59           5.0         3.0            8.0
AQUEDUCT RACETR Sunday    05:00-08:59           4.0         4.0            8.0
190 ST          Friday    01:00-04:59           5.0         2.0            7.0
HUNTS POINT AV  Monday    09:00-12:59           5.0         1.0            6.0
AQUEDUCT N.COND Saturday  21:00-00:59           3.0         3.0            6.0
WILSON AV       Thursday  21:00-00:59           1.0         5.0            6.0
80 ST           Saturday  09:00-12:59           3.0         2.0            5.0
BROAD CHANNEL   Sunday    01:00-04:59           3.0         2.0            5.0
BEACH 105 ST    Sunday    05:00-08:59           2.0         3.0            5.0
BROAD CHAN